In [1]:
from ipynb.fs.full.Infer_propagation_path_functions import *

from pymongo import MongoClient

In [2]:
client = MongoClient('mongodb://localhost:27017/')

db = client.get_database('tweets')

In [3]:
COLLECTION_NAME = 'data_true_5'
tweets = db[COLLECTION_NAME].find({})
followers = db['followers'].find({})

tweet_user_ids = []
for tweet in tweets:
    tweet_user_ids.append(tweet['user']['id'])

tweet_user_ids

[2269308902,
 378566529,
 847811211752243200,
 1349824160931844097,
 19753565,
 1265693243540484096,
 1359293410847383558,
 1349824160931844097,
 936333034403614722,
 33941015,
 1141392338822057985,
 3402382763,
 780460754910732288,
 1239295184644575233,
 2261025822,
 2288847752,
 2939862334,
 305012553,
 1206374923272364032,
 1388561251652050946,
 1184042986868490240,
 1219748687947149313,
 895367076973613056,
 2674158176,
 1714283762,
 1076489651022712833,
 2913568455,
 2890852846,
 1361734658367639552,
 1266376183937032192,
 1319131856105840641,
 1070009849667293184,
 1290679963444805643,
 1663787652,
 1261798762579664901,
 1200877628154142723,
 1362028461523689480,
 944751372175642624,
 1322220530305540096,
 27550771,
 318680721,
 1093158409690583048,
 819018783960535040,
 1246303759526854657,
 351302170,
 1114273249209221122,
 3330428663,
 1132370788747816962,
 1138852762463154177,
 188251099,
 320020660,
 2268959855,
 1022212324676001792,
 782950786858901504,
 1350679967101841409

In [23]:
'''
has a root user 1
root user has followers 1
If a root user followers intersects with tweets, they are cascade depth one users 1
if a cascade depth one user followers intersects with tweets, they are cascade depth two users
...
if a cascade depth n-1 user followers intersects with tweets, they are cascade depth n users

Lets say parent user publish a tweet and the child re-tweet that exact tweet.
child timestamp must be after the parent's time stamp
If child has many parents, parent with the most recent time stamp will be the owner of the child tweet  
'''
tweets = db[COLLECTION_NAME].find({'retweeted_status.user.id': 780460754910732288})
#get root tweet
root_tweet = set()
for tweet in tweets:
    try:
        root_tweet.add(tweet['retweeted_status']['user']['id'])
    except KeyError as e:
        continue
print(root_tweet)

tweets = db[COLLECTION_NAME].find({'retweeted_status.user.id': 780460754910732288})

# Create root node user_id, created_at

if len(root_tweet) == 1:
    for tweet in tweets:
        try:
            user_id = tweet['retweeted_status']['user']['id']
            created_at = tweet['retweeted_status']['created_at']
        except KeyError as e:
            continue
            
    print(user_id, created_at)

    root = Node(name = user_id, created_at = created_at, difference_seconds =  0)
    print(root.name)

    follower = db['followers'].find_one({'user_id' : root.name})
    try:
        print(follower['user_id'])
    except:
        print('None, DB does not have the user: ', root.name)
        
    #get depth 1 users

    if follower['user_id'] == root.name:
        root_followers = follower['followers']           

    root_followers

    tweets = db[COLLECTION_NAME].find({'retweeted_status.user.id': 780460754910732288})
    followers = db['followers'].find({})

    tweet_user_ids = []
    for tweet in tweets:
        tweet_user_ids.append(tweet['user']['id'])

    tweet_user_ids

    cascade_depth_one_user_ids = list(set(root_followers) & set(tweet_user_ids))

    len(cascade_depth_one_user_ids)

    tweets = db[COLLECTION_NAME].find({'retweeted_status.user.id': 780460754910732288})
    followers = db['followers'].find({})

    # add depth 1 users to the tree
    for user_id in cascade_depth_one_user_ids:
        tweets = db[COLLECTION_NAME].find({'retweeted_status.user.id': 780460754910732288})
        for tweet in tweets:
            if tweet['user']['id'] == user_id:
                print(tweet['created_at'])
                difference = difference_datetime(tweet['created_at'], root.created_at).total_seconds()
                print(difference)
                globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = root)


    for pre, fill, node in RenderTree(root):
        print("%s user_id: %s, created_at: %s, difference_seconds: %s" % (pre, node.name, node.created_at, node.difference_seconds))
else:
    multiple_root_users(root_tweet)
#     raise('Cascade has more than one root user')

{780460754910732288}
780460754910732288 Wed May 12 14:38:03 +0000 2021
780460754910732288
780460754910732288
Thu May 13 05:28:57 +0000 2021
53454.0
Wed May 12 15:31:19 +0000 2021
3196.0
Wed May 12 14:43:52 +0000 2021
349.0
Wed May 12 23:17:41 +0000 2021
31178.0
Wed May 12 18:30:53 +0000 2021
13970.0
Wed May 12 14:48:57 +0000 2021
654.0
Wed May 12 18:24:39 +0000 2021
13596.0
Wed May 12 16:11:03 +0000 2021
5580.0
Thu May 13 02:56:32 +0000 2021
44309.0
Wed May 12 15:17:46 +0000 2021
2383.0
Wed May 12 15:28:06 +0000 2021
3003.0
Wed May 12 20:26:25 +0000 2021
20902.0
Wed May 12 15:00:40 +0000 2021
1357.0
Wed May 12 15:55:06 +0000 2021
4623.0
Wed May 12 15:34:46 +0000 2021
3403.0
Thu May 13 11:36:04 +0000 2021
75481.0
Wed May 12 19:15:14 +0000 2021
16631.0
Wed May 12 15:45:23 +0000 2021
4040.0
Wed May 12 15:51:55 +0000 2021
4432.0
Wed May 12 17:46:23 +0000 2021
11300.0
Wed May 12 15:30:26 +0000 2021
3143.0
Thu May 13 02:51:20 +0000 2021
43997.0
Wed May 12 23:41:35 +0000 2021
32612.0
Wed May 

In [22]:
# export tree
from anytree.exporter import JsonExporter, DictExporter
exporter = DictExporter()
data = exporter.export(root)
with open(COLLECTION_NAME + '_tree.json', 'w') as fp:
    json.dump(data, fp)    

In [19]:
# import tree
from anytree.importer import DictImporter
from anytree import RenderTree
importer = DictImporter()
with open(COLLECTION_NAME + '_tree.json', 'r') as fp:
    data = fp.readline()
data = json.loads(data)
root = importer.import_(data)
print(RenderTree(root))

AnyNode(created_at='Wed May 12 14:38:03 +0000 2021', difference_seconds=0, name=780460754910732288)
├── AnyNode(created_at='Wed May 12 14:47:41 +0000 2021', difference_seconds=578.0, name=114503680)
├── AnyNode(created_at='Wed May 12 14:43:52 +0000 2021', difference_seconds=349.0, name=156134917)
├── AnyNode(created_at='Wed May 12 23:17:41 +0000 2021', difference_seconds=31178.0, name=1354082695622647808)
├── AnyNode(created_at='Wed May 12 18:24:39 +0000 2021', difference_seconds=13596.0, name=1362065164657451010)
├── AnyNode(created_at='Thu May 13 02:56:32 +0000 2021', difference_seconds=44309.0, name=1333520743926296579)
├── AnyNode(created_at='Wed May 12 16:11:03 +0000 2021', difference_seconds=5580.0, name=425531421)
├── AnyNode(created_at='Thu May 13 10:50:34 +0000 2021', difference_seconds=72751.0, name=1377626558425403395)
├── AnyNode(created_at='Wed May 12 15:28:06 +0000 2021', difference_seconds=3003.0, name=876631893952704512)
├── AnyNode(created_at='Wed May 12 20:26:25 +0000

In [24]:
remove_duplicates(root)

0
0


In [25]:
for i in range(2,10):
    add_depth_nodes(root, i, tweet_user_ids, db, COLLECTION_NAME)
    remove_duplicates(root)

[Node('/780460754910732288/1078871585035976709', created_at='Thu May 13 05:28:57 +0000 2021', difference_seconds=53454.0), Node('/780460754910732288/834896559280119810', created_at='Wed May 12 15:31:19 +0000 2021', difference_seconds=3196.0), Node('/780460754910732288/156134917', created_at='Wed May 12 14:43:52 +0000 2021', difference_seconds=349.0), Node('/780460754910732288/1354082695622647808', created_at='Wed May 12 23:17:41 +0000 2021', difference_seconds=31178.0), Node('/780460754910732288/1165123935995912192', created_at='Wed May 12 18:30:53 +0000 2021', difference_seconds=13970.0)]
 user_id: 780460754910732288, diff_s: 0
├──  user_id: 1078871585035976709, diff_s: 53454.0
├──  user_id: 834896559280119810, diff_s: 3196.0
├──  user_id: 156134917, diff_s: 349.0
├──  user_id: 1354082695622647808, diff_s: 31178.0
├──  user_id: 1165123935995912192, diff_s: 13970.0
├──  user_id: 1183019592446169088, diff_s: 654.0
├──  user_id: 1362065164657451010, diff_s: 13596.0
├──  user_id: 42553142

In [5]:
users= [1391686780798210052, 16640775, 287603080, 5392522, 18728203, 701725963, 318680721, 19201304, 21110041, 1138852762463154177, 847811211752243200, 14838173, 2261025822, 22705952, 18082466, 1245158946, 24736682, 900085327863218176, 428333, 1268569059731353600, 145692463, 23674801, 1381303367071305729, 17525171, 1153391118773604352, 25817278, 767609351037214720, 255801799, 126051146, 15791186, 18908628, 1359293410847383558, 780460754910732288, 14221917, 444054117, 2269308902, 14697575, 20562924, 57443437, 2828568811, 35773039, 241972592, 429112432, 931039604, 1339526904, 23447418, 4170491, 20087934]

In [6]:
for user in users:
    print(user, get_followers_len(user, db))

1391686780798210052 0
16640775 0
287603080 0
5392522 0
18728203 0
701725963 383374
318680721 0
19201304 0
21110041 0
1138852762463154177 0
847811211752243200 0
14838173 0
2261025822 0
22705952 0
18082466 0
1245158946 0
24736682 0
900085327863218176 0
428333 0
1268569059731353600 0
145692463 0
23674801 0
1381303367071305729 0
17525171 0
1153391118773604352 0
25817278 0
767609351037214720 0
255801799 0
126051146 0
15791186 0
18908628 0
1359293410847383558 0
780460754910732288 0
14221917 0
444054117 0
2269308902 0
14697575 0
20562924 0
57443437 0
2828568811 0
35773039 0
241972592 0
429112432 0
931039604 0
1339526904 0
23447418 0
4170491 0
20087934 0


In [7]:
for user in users:
    print(user, get_cascade_size(user, COLLECTION_NAME, db))

1391686780798210052 1
16640775 3
287603080 1
5392522 24
18728203 6
701725963 30
318680721 1
19201304 1
21110041 1
1138852762463154177 1
847811211752243200 10
14838173 1
2261025822 6
22705952 5
18082466 1
1245158946 1
24736682 2
900085327863218176 2
428333 171
1268569059731353600 1
145692463 1
23674801 1
1381303367071305729 1
17525171 101
1153391118773604352 5
25817278 2
767609351037214720 2
255801799 1
126051146 1
15791186 2
18908628 1
1359293410847383558 5
780460754910732288 1093
14221917 1
444054117 2
2269308902 5
14697575 11
20562924 1
57443437 1
2828568811 1
35773039 18
241972592 3
429112432 1
931039604 1
1339526904 1
23447418 3
4170491 2
20087934 3


In [8]:
# Check if the text is same
db[COLLECTION_NAME].distinct('retweet_count')

[0, 1, 2, 3, 4, 5, 6, 8, 11, 12, 18, 21, 26, 30, 120, 207, 1229]

In [9]:
db[COLLECTION_NAME].distinct('retweeted_status.retweet_count')

[1, 2, 3, 4, 5, 6, 8, 11, 12, 18, 21, 26, 30, 120, 207, 1229]

In [10]:
db[COLLECTION_NAME].distinct('retweeted_status.quoted_status.retweet_count')

[]

In [11]:
# Check if the text is same
cache = db[COLLECTION_NAME].distinct('retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweet_count':x})['text'])
    print('*'*5)

[26, 30, 120, 207, 1229]
Ellen DeGeneres has announced that her daytime talk show will end in 2022.

Recent scandals suggest that the host’s… https://t.co/wW9tZTbGJZ
*****
Ellen DeGeneres has announced her daytime talk show will end in 2022 after 19 seasons — citing a desire for new cha… https://t.co/9zdG84W3wC
*****
“The Ellen DeGeneres Show” will officially end in 2022. DeGeneres has hosted the talk show since 2003, meaning it w… https://t.co/ZKgZFLGXvf
*****
"The Ellen DeGeneres Show," currently in season 18, will end in 2022 after season 19, according to a report in The… https://t.co/oC6mwubwqE
*****
The Ellen Show will end in 2022.

(Source: https://t.co/3oBLLyR4CX) https://t.co/ISvMw8C7is
*****


In [12]:
cache = db[COLLECTION_NAME].distinct('retweeted_status.retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweeted_status.retweet_count':x})['id'])
    print('*'*5)

[26, 30, 120, 207, 1229]
1392522669145403393
*****
1392572851409063936
*****
1392491210963525636
*****
1392499938244837383
*****
1392489280996581380
*****


In [ ]:
cache = db[COLLECTION_NAME].distinct('retweeted_status.quoted_status.retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweeted_status.quoted_status.retweet_count':x})['retweeted_status.quoted_status.id'])
    print('*'*5)

In [ ]:
ROOT_USER = 780460754910732288